# Lesson 24. Using methods as if they were functions

In [2]:
def doubleMethod(i: Int) = i * 2
List(1,2,3).map(doubleMethod)

doubleMethod: (i: Int)Int
res1: List[Int] = List(2, 4, 6)


## How to manually convert a method to a function
Attempting to show the value of a method in the REPL will give a handy error message to remember this by:

In [3]:
def triple(i: Int) = i * 3

triple: (i: Int)Int


In [4]:
triple

<console>: 28: error: missing argument list for method triple

As the error message says, follow the method with a _ (also assign it to capture the value in this case):

In [5]:
val tripleFn = triple _

tripleFn: Int => Int = $Lambda$2000/0x0000000840be9040@3c8087c7


In [6]:
tripleFn(1)

res3: Int = 3


In [7]:
val x = List(1,2,3)
x.map(tripleFn)

x: List[Int] = List(1, 2, 3)
res4: List[Int] = List(3, 6, 9)


In [8]:
x.map(triple)

res5: List[Int] = List(3, 6, 9)


## Example where methods will fail unless converted into a function:

In [9]:
def double(i: Int) = i * 2
def triple(i: Int) = i * 3

double: (i: Int)Int
triple: (i: Int)Int


In [10]:
val functions = Map(
    "2x" -> double,
    "3x" -> triple)

<console>: 28: error: missing argument list for method double

Fixed by converting to functions:

In [11]:
val functions = Map(
    "2x" -> double _,
    "3x" -> triple _)

functions: scala.collection.immutable.Map[String,Int => Int] = Map(2x -> $Lambda$2060/0x0000000840c2d040@2204b302, 3x -> $Lambda$2061/0x0000000840c2e040@537438d6)


Retrieving and 

In [12]:
functions("2x")(3)

res6: Int = 6


# Lesson 25. How to write functions that take functions as input parameters
#### Terminology note -- FIP - function input parameter
## Introduction
Like in C# you are defining the signature of the function you want to accept when dealing with HOFs:

In [13]:
def sayHello(callback: () => Unit) {
    callback()
}

sayHello: (callback: () => Unit)Unit


Since this is a no parameter function with no result its basically looking for a func to run for side effects. As evidenced by the example for using it:

In [14]:
def helloTrite(): Unit = { println("Hello, Trite") }

sayHello(helloTrite)

Hello, Trite


helloTrite: ()Unit


## More parameters, everywhere
Decent example for taking a FIP with some non-function values:

In [15]:
def executeAndPrint(f: (Int, Int) => Int, x: Int, y: Int): Unit =
    println(f(x, y))

def sum(x: Int, y: Int) = x + y
def multiply(x: Int, y: Int) = x * y

executeAndPrint(sum, 3, 11)
executeAndPrint(multiply, 3, 9)

14
27


executeAndPrint: (f: (Int, Int) => Int, x: Int, y: Int)Unit
sum: (x: Int, y: Int)Int
multiply: (x: Int, y: Int)Int


## Taking multiple functions as input parameters
Define a function that takes:
 * (Int, Int) => Int ((x2))
 * Int ((x2))
 * The Ints will be passed to the two FIPs
 * Return the results from the functions as a tuple

#### Attempt:

In [16]:
def myFunc(f1: (Int, Int) => Int, f2: (Int, Int) => Int, x: Int, y: Int): (Int, Int) =
    (f1(x, y), f2(x, y))

def sum(x: Int, y: Int) = x + y
def multiply(x: Int, y: Int) = x * y

myFunc(sum, multiply, 9, 27)

myFunc: (f1: (Int, Int) => Int, f2: (Int, Int) => Int, x: Int, y: Int)(Int, Int)
sum: (x: Int, y: Int)Int
multiply: (x: Int, y: Int)Int
res9: (Int, Int) = (36,243)


#### Following along with the book:

In [17]:
def execTwoFunctions(f1:(Int, Int) => Int,
                     f2:(Int, Int) => Int,
                     a: Int,
                     b: Int): Tuple2[Int, Int] =
    (f1(a,b), f2(a,b))

def sum(x: Int, y: Int) = x + y
def multiply(x: Int, y: Int) = x * y

execTwoFunctions(sum, multiply, 2, 10)

execTwoFunctions: (f1: (Int, Int) => Int, f2: (Int, Int) => Int, a: Int, b: Int)(Int, Int)
sum: (x: Int, y: Int)Int
multiply: (x: Int, y: Int)Int
res10: (Int, Int) = (12,20)


# 26. How to write a 'map' function
The author outlines a standardized way to approach writing HOFs
#### 1. Accurately state the problem as a sentence
"I want to write a map function that can be used to apply other functions to each element in a List[Int] that it's given."
#### 2. Sketch the function signature:

In [18]:
def map[A,B](f: A => B, list: Seq[A]): Seq[B] = ???
42 // returning a value to prevent the cell output vomiting everywhere

map: [A, B](f: A => B, list: Seq[A])Seq[B]
res11: Int = 42


#### 3. Figure out the function body:

In [19]:
def map[A,B](f: A => B, list: Seq[A]): Seq[B] = {
    for {
        x <- list
    } yield f(x)
}

map: [A, B](f: A => B, list: Seq[A])Seq[B]


## Exercise: write a filter function


In [22]:
def filter[A](f: A => Boolean, list: Seq[A]): Seq[A] = {
    for {
        x <- list if f(x)
    } yield x
}

def isEven(x: Int): Boolean =
    x % 2 == 0

filter(isEven, Range(1,21))

filter: [A](f: A => Boolean, list: Seq[A])Seq[A]
isEven: (x: Int)Boolean
res14: Seq[Int] = Vector(2, 4, 6, 8, 10, 12, 14, 16, 18, 20)


In [21]:
Range(1,20)

res13: scala.collection.immutable.Range = Range 1 until 20


# 27. How to use by-name parameters
Sometimes the goal of accepting a function is to pass a larger block of code, which feels like it would be wrong to call that a HOF for some reason. Since there's apparently a special name for that it hopefully means that feeling was correct and a good sign.
## This technique can be referred to as either a "by-name" or a "call by-name" parameter apparently
The alternative being a "by-value" parameter, which in the case of a function means a lazily evaluated function (I think). The "by-name" parameter will apparently be evaluated each time it is used? Re-read this chapter and spots around Loc 2750 (page 251) in the book to clarify this later if needed.
## Example: Creating a timer:

In [24]:
def timer[A](blockOfCode: => A) = {
    val startTime = System.nanoTime
    val result = blockOfCode
    val stopTime = System.nanoTime
    val delta = stopTime - startTime
    (result, delta/1000000d)
}

val (result, time) = timer {
    Thread.sleep(500)
    println("done")
    42
}

done


timer: [A](blockOfCode: => A)(A, Double)
result: Int = 42
time: Double = 500.2065


In [25]:
def test[A](codeBlock: => A) = {
    println("before 1st codeBlock")
    val a = codeBlock
    println(a)
    Thread.sleep(10)
    
    println("before 2nd codeBlock")
    val b = codeBlock
    println(b)
    Thread.sleep(10)
    
    println("before 3rd codeBlock")
    val c = codeBlock
    println(c)
}

test(System.currentTimeMillis)

before 1st codeBlock
1632686718787
before 2nd codeBlock
1632686718797
before 3rd codeBlock
1632686718808


test: [A](codeBlock: => A)Unit


## Why care? This seems like reason enough...
You can provide just the function body instead of needing to also provide the function signature in cases like the one below.
#### If starting with a function like this:

In [27]:
val assertionsEnabled = true

def myAssert(predicate: () => Boolean) =
    if (assertionsEnabled && !predicate())
        throw new AssertionError

assertionsEnabled: Boolean = true
myAssert: (predicate: () => Boolean)Unit


#### Then you need to use it like this:

In [30]:
myAssert(() => 5 > 3)

#### By starting like this instead:

In [32]:
def byNameAssert(predicate: => Boolean) =
    if (assertionsEnabled && !predicate)
        throw new AssertionError

byNameAssert: (predicate: => Boolean)Unit


#### Then you can use it in a much more natural way. Wish C# would add something similar:

In [33]:
byNameAssert(5 > 3)